<h1>Datascience Capstone Project</h1>

In this capstone project, hopefully I will learn new skill related location data and how to use it develop cool applciations.

<h1>Assignment 1</h1>

In [1]:
#import numpy and panda
import numpy as np 
import pandas as pd

In [2]:
#import requests for getting the HTML contents of the website and lxml.html for parsing the relevant field
# references : https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
import requests
import lxml.html as lh

In [3]:
#Create a handle, page, to handle the contents of the website
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#import beautiful soup
from bs4 import BeautifulSoup

#Store the contents of the website under doc
res = requests.get(url)

In [4]:
#break the cell content into PostalCode, Borough, and Neighborhood
def split_text(txt):
    #print(txt)
    txt = txt.replace('\n','')  #remove all \n
    x = txt[0:3]       # the the postal code
    y = txt[3:len(txt)] 
    z = y.split('(') # split borough and neighborhood
    z.insert(0,x)  # put all into array
    z=np.array(z)
    return(z)


In [23]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Postal Code,Borough,Neighborhood


In [24]:
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
    
#iteration through the cell
intial=0;

for row in table.find_all('tr'):
    for cell in row.find_all('td'):
       s_text = split_text(cell.text)
       #print(s_text)
       #print(len(s_text))
       if s_text[1] != "Not assigned":
          postalcode = s_text[0]
          borough = s_text[1]
          if (len(s_text)==2):
             neighborhood_name = s_text[1]   
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough,
                                                   'Neighborhood': neighborhood_name}, ignore_index=True)
          elif (len(s_text)==3):
             neighborhood_name = s_text[2]
             neighborhood_name = neighborhood_name[0:(len(neighborhood_name) - 1)]      #remove ')'        
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough, 
                                                   'Neighborhood': neighborhood_name}, ignore_index=True)   
          elif (len(s_text) > 3):
             neighborhood_name = s_text[2]
             neighborhood_name = neighborhood_name[0:(len(neighborhood_name) - 1)]      #remove ')'        
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough, 
                                                   'Neighborhood': neighborhood_name}, ignore_index=True) 
            

In [25]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [26]:
neighborhoods.shape

(103, 3)

<h1>Assignment 2</h1>

In [27]:
#Read the coordinate - as geocoder - cannot be imported
geo_loc = pd.read_csv("http://cocl.us/Geospatial_data") 


In [28]:
# Preview the first 5 lines of the loaded data 
geo_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
geo_loc.shape

(103, 3)

In [34]:
result = neighborhoods.sort_values(['Postal Code'], ascending=1, axis=0)

In [35]:
result.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,Malvern / Rouge
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
18,M1E,Scarborough,Guildwood / Morningside / West Hill
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [36]:
result.shape

(103, 3)

In [37]:
sorted_neigh = result.reset_index(drop=True) #drop parameter to avoid the old index being added as a column
sorted_neigh.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [42]:
#merge neighboorhood and geospatial data
df_neigh_loc = pd.merge(sorted_neigh, geo_loc, on='Postal Code')

In [43]:
df_neigh_loc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
df_neigh_loc.shape

(103, 5)

<h1>Assignment 3 <h1>